# Recategorize score
 Rescore model probabilities using Buckets from train. Saves results to specified database table

## Load libraries

In [53]:
pip install tabgan==1.3.3

Note: you may need to restart the kernel to use updated packages.


In [54]:
!pip install lightgbm

In [55]:
pip install scikit-plot

Note: you may need to restart the kernel to use updated packages.


In [56]:
%pip install scipy==1.11.4

Note: you may need to restart the kernel to use updated packages.


In [57]:
pip install --upgrade pandas "dask[complete]"

Note: you may need to restart the kernel to use updated packages.


In [58]:
%pip install scikit-learn==1.1.3

Note: you may need to restart the kernel to use updated packages.


In [59]:
## librerias requeridas
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import calendar
import re
from sklearn.metrics import confusion_matrix 
from sklearn.model_selection import train_test_split 
from sklearn.tree import DecisionTreeClassifier 
from sklearn.metrics import accuracy_score 
from sklearn.metrics import classification_report 
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score
from scikitplot.metrics import plot_roc
from sklearn.metrics import roc_auc_score
from scikitplot.metrics import plot_lift_curve
from scikitplot.helpers import binary_ks_curve 
from scikitplot.metrics import plot_ks_statistic
from scikitplot.helpers import cumulative_gain_curve
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from lightgbm import LGBMClassifier
from datetime import datetime, timedelta

import pyarrow as pa
import pyarrow.parquet as pq

In [60]:
import sklearn
print(sklearn.__version__)

1.1.3


In [61]:
pd.set_option('display.max_columns', 500)
pd.set_option('display.memory_usage', True)

## Load model and data to evaluate

Load variables names from file

In [62]:
import os
os.getcwd()

'/root/viamericas-dev/Analytics/5.Fraud'

In [63]:
import os
import joblib
# load model
modelo = joblib.load("ModeloAWSRetrain.pkl") 
variables = open('./ModeloBiteRetrain.txt','r')

/opt/conda/lib/python3.10/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator LabelEncoder from version 1.4.2 when using version 1.1.3. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [64]:
modelo

LGBMClassifier(colsample_bytree=0.84, importance_type='gini',
               learning_rate=0.07, max_depth=20, min_child_weight=0.0026,
               n_estimators=520, num_leaves=30, objective='binary',
               scale_pos_weight=0.003, subsample=1)

## Veo si puedo sacar los nombres directo del modelo q cargue
## Veo como cargar el punto de corte

In [65]:
variables = open('./ModeloBiteRetrain.txt','r')
var_input = variables.read()

#1er prueba
var_input = ['branch_minutes_since_last_transaction',
    'branch_trans_3m',  'branch_has_fraud',  
    'branch_trans_40min', 'branch_trans_10min', 'cash_pick_up_40min', 
    'location_nro_fraud', 'sender_trans_3m', 
    'sender_nro_fraud','01_isMexico', 
    '01_sender_branch_state', '01_var_range_hist', 
    '01_branch_fraud', '01_location_fraud', '01_sender_fraud', 
    '01_isCashPick', '01_isBankDep', 
    'sender_days_to_last_transaction_more7m', 
    '01_receiver_transaction_count','01_sender_sending_days',
    '01_branch_working_days', '01_net_amount_receiver', 
    '01_sender_minutes_since_last_transaction_2days', 
    '01_sender_days_to_last_transaction_365', 
    '01_sender_days_to_last_transaction_7m', '01_hour_receiver']

In [66]:
#2da prueba
var_input = ['branch_minutes_since_last_transaction',
    'branch_trans_3m',  'branch_has_fraud',  
    'branch_trans_40min', 'branch_trans_10min', 'cash_pick_up_40min', 
    'location_nro_fraud', 'sender_trans_3m', 
    'sender_nro_fraud','01_isMexico', 
    '01_sender_branch_state', '01_var_range_hist', 
    '01_branch_fraud', '01_location_fraud',
    '01_isCashPick', '01_isBankDep', 
    'sender_days_to_last_transaction_more7m', 
    'receiver_transaction_count','01_sender_sending_days',
    'branch_working_days', '01_net_amount_receiver', 
    'sender_minutes_since_last_transaction_2days', 
    'sender_days_to_last_transaction_365', 
    'sender_days_to_last_transaction_7m', '01_hour_receiver']

In [67]:
s3_url = f"viamericas-datalake-dev-us-east-1-283731589572-raw/FraudModel/Data4Model/TargetTodosRetrain"
#tes = pd.read_parquet(f"s3://{s3_url}/Data2024/Data2024.parquet")

tes = pd.read_parquet(f"s3://{s3_url}/Test/Test.parquet")

## Variable list for input
X = tes[var_input]
Y = tes[['target']]

dias_target = ['day_name_receiver_0', 'day_name_receiver_1',
       #'day_name_receiver_2', 'day_name_receiver_3',
       #'day_name_receiver_4', 'day_name_receiver_5',
       'day_name_receiver_6','target']

testeo =  tes.copy()
Y_target = pd.DataFrame(tes[dias_target])



In [68]:
#s3_url = f"viamericas-datalake-dev-us-east-1-283731589572-raw/FraudModel/Data4Model/TargetTodos"
#tes = pd.read_parquet(f"s3://{s3_url}/Data2024/Data2024.parquet")

#testeo = pd.concat([testeo, tes], axis = 0, ignore_index=True)
#Y_target = pd.DataFrame(pd.concat([Y_target, tes[dias_target]], ignore_index=True)) 

## Apply model

In [69]:
testeo2 = testeo[var_input]

#Rename variables
testeo2.rename(columns={'01_isMexico':'a_isMexico',
    '01_sender_branch_state':'a_sender_branch_state', 
    '01_var_range_hist':'a_var_range_hist', 
    '01_branch_fraud':'a_branch_fraud', 
    '01_location_fraud':'a_location_fraud',
    '01_isCashPick':'a_isCashPick', 
    '01_isBankDep':'a_isBankDep', 
    '01_sender_sending_days':'a_sender_sending_days',
    '01_net_amount_receiver':'a_net_amount_receiver' }, inplace=True)

/tmp/ipykernel_1425/1246366372.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  testeo2.rename(columns={'01_isMexico':'a_isMexico',


In [70]:

# Prediciendo la variable objetivo para el conjunto de datos de prueba 'X_TESTEO' utilizando el clasificador LightGBM
testeo_data_pred_rf = modelo.predict(testeo2)

# Prediciendo las probabilidades de clase para cada observación en 'X_TESTEO' utilizando el clasificador LightGBM
probab_rf = modelo.predict_proba(testeo2)

# Eliminando la primera columna (se asume que son las probabilidades de la clase negativa) de las probabilidades predichas
# Se asume que la segunda columna contiene las probabilidades para la clase positiva
score_rf = np.delete(probab_rf, np.s_[0], axis=1)


In [71]:

# Creando una copia del DataFrame 'Y_TESTEO' para preservar los datos originales
Y_c = Y_target.copy()

# Agregando una nueva columna 'preds_rf' al DataFrame copiado con las predicciones del clasificador LightGBM
Y_c['preds_rf'] = testeo_data_pred_rf

# Agregando una nueva columna 'score_rf' al DataFrame copiado con las probabilidades de la clase positiva
Y_c['score_rf'] = score_rf

In [72]:
pd.DataFrame(confusion_matrix(Y_c['target'], Y_c['preds_rf']))

,0,1
0,5870909,0
1,51,1


## Transformo los datos para subirlos a Athena

In [73]:
Buckets = pd.read_csv('./BucketsModeloAWSRetrain.csv')

In [74]:
# Create bins from reference dataframe
pd.options.display.float_format = "{:,.15f}".format
# Convert data types of 'Min_Value' and 'Max_Value' to numeric
Buckets['min'] = pd.to_numeric(Buckets['min'])
Buckets['max'] = pd.to_numeric(Buckets['max'])
Y_c['score_rf'] = pd.to_numeric(Y_c['score_rf'])

#bins = np.sort(np.append(Buckets['min'], Buckets['max'].iloc[-1]))
bins = np.sort(np.append(Buckets['min'], np.inf))  

labels = np.sort(Buckets['final_score'])
#labels = np.sort(1000-Buckets['FRAUD_DECILE'])

# Recode scores using pd.cut()
Y_c['score_AWS'] = pd.cut(Y_c['score_rf'], bins=bins, labels=labels) ##, include_lowest=True, right=False)
#Y_c['Recode'] = Y_c['Recode'].astype('Int32')

Y_c.head()

,day_name_receiver_0,day_name_receiver_1,day_name_receiver_6,target,preds_rf,score_rf,score_AWS
0,False,False,False,False,False,0.000000000000422,0.079000000000000
1,False,False,True,False,False,0.000000000014773,0.618000000000000
2,True,False,False,False,False,0.000000000018106,0.648000000000000
3,False,True,False,False,False,0.000000000090659,0.825000000000000
4,False,False,False,False,False,0.000000000001800,0.269000000000000


In [75]:
Y_c['pred_corte_01'] = Y_c['score_AWS']>0.998 

In [76]:
Y_c['pred_corte_01'].value_counts()

pred_corte_01
False    5866724
True        4237
Name: count, dtype: int64

In [77]:
#See false positives by month
tes_corte = testeo[Y_c['pred_corte_01']==True]
Y_c_corte = Y_c[Y_c['pred_corte_01']==True]

pd.crosstab(tes_corte['date_receiver'].dt.day, Y_c_corte['target'])

target,False,True
date_receiver,,
1,141,0
2,121,0
3,132,0
4,144,0
5,124,0
6,134,0
7,154,0
8,117,0
9,145,0


In [78]:
tes_corte[tes_corte['date_receiver'].dt.day==30]

,id_branch,id_receiver,date_receiver,id_location,id_payer,id_sender_global,net_amount_receiver,id_country_receiver_claim,id_state_receiver_claim,id_state,branch_working_days,sender_sending_days,sender_days_to_last_transaction,id_country,fraud_classification,sender_minutes_since_last_transaction,branch_minutes_since_last_transaction,sender_days_since_last_transaction,branch_trans_3m,sender_state,branch_has_fraud,receiver_has_fraud,branch_trans_40min,branch_trans_10min,cash_pick_up_40min,location_nro_fraud,sender_trans_3m,range_hist,sender_nro_fraud,receiver_transaction_count,mes,hour_receiver,01_isMexico,01_sender_branch_state,01_var_range_hist,01_receiver_fraud,01_branch_fraud,01_location_fraud,01_sender_fraud,01_isCashPick,01_isBankDep,sender_days_to_last_transaction_365,sender_days_to_last_transaction_more365,sender_days_to_last_transaction_7m,sender_days_to_last_transaction_more7m,sender_minutes_since_last_transaction_2days,sender_minutes_since_last_transaction_1day,01_net_amount_receiver,01_receiver_transaction_count,01_sender_sending_days,01_sender_minutes_since_last_transaction_2days,01_sender_minutes_since_last_transaction_1day,01_sender_days_to_last_transaction_365,01_sender_days_to_last_transaction_7m,01_hour_receiver,day_name_receiver_0,day_name_receiver_1,day_name_receiver_2,day_name_receiver_3,day_name_receiver_4,day_name_receiver_5,day_name_receiver_6,id_payout_A,id_payout_C,id_payout_D,id_payout_G,id_payout_M,id_payout_N,id_payout_O,id_payout_P,id_payout_S,id_payout_T,id_payout_X,targetR,targetP,target
2849727,A19459,15708,2024-05-30 16:05:40.693,OK0197,T239,11944826,980,MEX,MEX1,OK,1456,0,0,USA,0,0,186,0,"1,629.000000000000000",OK,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0,1,5,16,True,True,NaN,False,False,False,False,True,False,0,0,0,0,0,0,0.353790613718412,0.000000000000000,0.000000000000000,0.000000000000000,0.000000000000000,0.000000000000000,0.000000000000000,0.666666666666667,False,False,False,True,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False
2860809,A26586,474,2024-05-30 18:58:06.897,WI0267,T057,11945932,980,MEX,MEX12,WI,63,0,0,USA,0,0,61,0,419.000000000000000,WI,NaN,2.000000000000000,NaN,NaN,NaN,NaN,0,NaN,0,2,5,18,True,True,NaN,True,False,False,False,False,True,0,0,0,0,0,0,0.353790613718412,0.009900990099010,0.000000000000000,0.000000000000000,0.000000000000000,0.000000000000000,0.000000000000000,0.750000000000000,False,False,False,True,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False
2927995,A22856,14259,2024-05-30 21:00:24.550,NY1282,T283,11946739,1500,MEX,MEX18,NY,698,0,0,USA,0,0,15,0,"3,343.000000000000000",NY,4.000000000000000,NaN,3.000000000000000,NaN,1.000000000000000,4.000000000000000,0,NaN,0,1,5,21,True,True,NaN,False,True,True,False,False,True,0,0,0,0,0,0,0.541516245487365,0.000000000000000,0.000000000000000,0.000000000000000,0.000000000000000,0.000000000000000,0.000000000000000,0.875000000000000,False,False,False,True,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False
3028564,A26187,350,2024-05-30 16:15:26.387,CA1591,T232,11944875,985,MEX,MEX8,CA,90,0,0,USA,0,0,1442,0,304.000000000000000,CA,NaN,1.000000000000000,NaN,NaN,NaN,NaN,0,NaN,0,1,5,16,True,True,NaN,True,False,False,False,False,True,0,0,0,0,0,0,0.355595667870036,0.000000000000000,0.000000000000000,0.000000000000000,0.000000000000000,0.000000000000000,0.000000000000000,0.666666666666667,False,False,False,True,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False
3158049,A25260,703,2024-05-30 23:55:40.800,CA4593,T350,4332588,1000,MEX,MEX2,CA,265,0,0,USA,0,0,116,0,347.000000000000000,CA,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0,1,5,23,True,True,NaN,False,False,False,False,False,True,0,0,0,0,0,0,0.361010830324910,0.000000000000000,0.000000000000000,0.000000000000000,0.000000000000000,0.000000000000000,0.000000000000000,0.958333333333333,False,False,False,True,F

In [79]:
df = pd.concat([testeo[['id_branch','id_receiver']], Y_c[['score_AWS','pred_corte_01']]], axis=1) 
df = df.rename(columns={"pred_corte_01": "was_fraud_AWS"})

## Guardo en Athena

In [80]:
pip install awswrangler

Note: you may need to restart the kernel to use updated packages.


In [81]:
df

,id_branch,id_receiver,score_AWS,was_fraud_AWS
0,A14129,50443,0.079000000000000,False
1,A18495,46810,0.618000000000000,False
2,A17198,43991,0.648000000000000,False
3,A24478,2098,0.825000000000000,False
4,A23098,8119,0.269000000000000,False
...,...,...,...,...
5870956,A23150,23005,0.296000000000000,False
5870957,A21735,37795,0.573000000000000,False
5870958,A26420,246,0.990000000000000,False
5870959,A20807,10441,0.263000000000000,False


In [82]:
import awswrangler as wr
import pandas as pd
import boto3
import time
from time import sleep

# Reemplazar aca por el dataframe a guardar
df = df

wr.s3.to_parquet(
    df=df,
    path='s3://viamericas-datalake-dev-us-east-1-283731589572-analytics/ml_score_aws/',
    dataset=True,
    mode="overwrite"
)

#lambda_crawler
def wait_for_crawler_completion(crawler_name: str, poll_interval: int = 15) -> str:
	"""
	Waits until Glue crawler completes and
	returns the status of the latest crawl run.
	Raises AirflowException if the crawler fails or is cancelled.
	:param crawler_name: unique crawler name per AWS account
	:param poll_interval: Time (in seconds) to wait between two consecutive calls to check crawler status
	:return: Crawler's status
	"""
	client = boto3.client('glue')
	response = client.start_crawler(Name=crawler_name)
	failed_status = ['FAILED', 'CANCELLED']
	sleep(5)

	while True:
		crawler = client.get_crawler(Name=crawler_name)
		crawler_state = crawler['Crawler']['State']
		if 'LastCrawl' in crawler['Crawler']:
			if crawler_state == 'STOPPING' or crawler_state == 'READY':
				print("State: %s", crawler_state)
				print("crawler_config: %s", crawler)
				crawler_status = crawler['Crawler']['LastCrawl']['Status']
				if crawler_status in failed_status:
						raise Exception(f"Status: {crawler_status}")
				metrics = client.get_crawler_metrics(CrawlerNameList=[crawler_name])['CrawlerMetricsList'][0]
				print("Status: %s", crawler_status)
				print("Last Runtime Duration (seconds): %s", metrics['LastRuntimeSeconds'])
				print("Median Runtime Duration (seconds): %s", metrics['MedianRuntimeSeconds'])
				print("Tables Created: %s", metrics['TablesCreated'])
				print("Tables Updated: %s", metrics['TablesUpdated'])
				print("Tables Deleted: %s", metrics['TablesDeleted'])

				return crawler_status

			else:
					print("Polling for AWS Glue crawler: %s ", crawler_name)
					print("State: %s", crawler_state)

					metrics = client.get_crawler_metrics(CrawlerNameList=[crawler_name])[
							'CrawlerMetricsList'
					][0]
					time_left = int(metrics['TimeLeftSeconds'])

					if time_left > 0:
							print("Estimated Time Left (seconds): %s", time_left)
					else:
							print("Crawler should finish soon")

					time.sleep(poll_interval)
		else:
			if crawler_state == 'READY':
					print("State: %s", crawler_state)
					print("crawler_config: %s", crawler)
					crawler_status = crawler['Crawler']['LastCrawl']['Status']
					if crawler_status in failed_status:
							raise Exception(f"Status: {crawler_status}")
					metrics = client.get_crawler_metrics(CrawlerNameList=[crawler_name])['CrawlerMetricsList'][0]
					print("Status: %s", crawler_status)
					print("Last Runtime Duration (seconds): %s", metrics['LastRuntimeSeconds'])
					print("Median Runtime Duration (seconds): %s", metrics['MedianRuntimeSeconds'])
					print("Tables Created: %s", metrics['TablesCreated'])
					print("Tables Updated: %s", metrics['TablesUpdated'])
					print("Tables Deleted: %s", metrics['TablesDeleted'])

					return crawler_status

			else:
					print("Polling for AWS Glue crawler: %s ", crawler_name)
					print("State: %s", crawler_state)

					metrics = client.get_crawler_metrics(CrawlerNameList=[crawler_name])[
							'CrawlerMetricsList'
					][0]
					time_left = int(metrics['TimeLeftSeconds'])

					if time_left > 0:
							print("Estimated Time Left (seconds): %s", time_left)
					else:
							print("Crawler should finish soon")

					time.sleep(poll_interval)


CrawlerName = 'crw_fraud_model_input'
crawler_to_run = f'{CrawlerName}'
print('Running Crawler: ', crawler_to_run)
response = wait_for_crawler_completion(crawler_name=crawler_to_run)
print('Finish with status: ', response)

Running Crawler:  crw_fraud_model_input
Polling for AWS Glue crawler: %s  crw_fraud_model_input
State: %s RUNNING
Crawler should finish soon
Polling for AWS Glue crawler: %s  crw_fraud_model_input
State: %s RUNNING
Crawler should finish soon
Polling for AWS Glue crawler: %s  crw_fraud_model_input
State: %s RUNNING
Crawler should finish soon
Polling for AWS Glue crawler: %s  crw_fraud_model_input
State: %s RUNNING
Crawler should finish soon
State: %s STOPPING
crawler_config: %s {'Crawler': {'Name': 'crw_fraud_model_input', 'Role': 'sdlf-lakeformation-admin', 'Targets': {'S3Targets': [{'Path': 's3://viamericas-datalake-dev-us-east-1-283731589572-analytics/ml_score_aws/', 'Exclusions': []}], 'JdbcTargets': [], 'MongoDBTargets': [], 'DynamoDBTargets': [], 'CatalogTargets': [], 'DeltaTargets': [], 'IcebergTargets': [], 'HudiTargets': []}, 'DatabaseName': 'analytics', 'Classifiers': [], 'RecrawlPolicy': {'RecrawlBehavior': 'CRAWL_EVERYTHING'}, 'SchemaChangePolicy': {'UpdateBehavior': 'UPDATE